## LELEC2350 - Simulation of a dispersive propagation
### Mathieu Reniers (30322000) - Colin Renard (51002000)

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from numba import jit
from tqdm.autonotebook import trange, tqdm
%matplotlib qt

#### 1. Useful parameters & Constants

In [33]:
Nt = 200; Nz = int(1e4); Nf = 150       # Resolutions of the simulation

z = np.linspace(0,10,Nz)                # z-axis [m]

fc = 3e9                                # Central frequency [Hz]
df = 1e9                                # Deviation frequency [Hz]
f = np.linspace(fc-df,fc+df,Nf)         # Frequency axis [Hz]
t = np.linspace(0,100*(1/fc),Nt)        # Time axis [s]

w,wc = 2*np.pi*f, 2*np.pi*fc            # Pulsation axis & Central pulsation
sigma_w = 2*np.pi*0.2*df                # Standard deviation of pulsations

A = (1/(np.sqrt(2*np.pi)*sigma_w)) * np.exp(-(w-wc)**2/(2*sigma_w**2))     # Amplitude distribution over frequences (Gaussian)
mu = 4*np.pi*1e-7
epsilon = 8.85*1e-12
c = 3e8

#### 2. Non-dispersive case

In [31]:
k = w*np.sqrt(mu*epsilon)                          

@jit(nopython=True,error_model="numpy") # Optimization
def compute_s(z,t):
    s = np.zeros(len(z))
    for i in range(len(z)):
        s[i] = np.real(np.sum(A*np.exp(-1j*k*z[i])*np.exp(1j*w*t)))
    return s

In [32]:
s_non_dispersive = np.zeros((Nt,Nz))
for i in tqdm(range(Nt), "time"):
    s_non_dispersive[i] = compute_s(z,t[i])

time:   0%|          | 0/200 [00:00<?, ?it/s]

#### 3. Dispersive case

In [34]:
wc_disp = 2*np.pi * 2e9
k_disp = (1/c) * np.sqrt((w+wc_disp)) * np.sqrt((w-wc_disp))

@jit(nopython=True,error_model="numpy") # Optimization
def compute_s_disp(z,t):
    # Computes s for an array z at a particular time t
    s = np.zeros(len(z))
    for i in range(len(z)):
        s[i] = np.real(np.sum(A*np.exp(-1j*k_disp*z[i])*np.exp(1j*w*t)))
    return s

In [35]:
s_dispersive = np.zeros((Nt,Nz))
for i in tqdm(range(Nt), "time"):
    s_dispersive[i] = compute_s_disp(z,t[i])

time:   0%|          | 0/200 [00:00<?, ?it/s]

#### 4. Animation

In [43]:
fig, ax = plt.subplots()
ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.set_title("Propagation in a non-dispersive medium")
ax.grid()

line, = ax.plot(z, s_non_dispersive[0], label="Non dispersive medium")
line2, = ax.plot(z, s_dispersive[0], label="Dispersive medium")
ax.legend()

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    line.set_ydata(s_non_dispersive[i])
    line2.set_ydata(s_dispersive[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=50, repeat=True ,blit=False)
plt.show()

writer = animation.PillowWriter(fps=15,
                                 metadata=dict(artist='Me'),
                                 bitrate=1800)
ani.save('scatter.gif', writer=writer)